In [1]:
import os, sys, time
from time import sleep
from datetime import *
from pathlib import Path
import fnmatch, glob, shutil
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import tempfile, logging
from zipfile import ZipFile

In [2]:
from PyPDF2 import PdfFileWriter, PdfFileReader

In [3]:
# SETUP LOGGER
try: #{
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s : %(message)s',
                        datefmt='%Y-%m-%d-%H%M%S',
                        filemode='a')
#}
except: #{
    errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
    errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
    errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    typeE = str("TYPE : " + str(exc_type))
    fileE = str("FILE : " + str(fname))
    lineE = str("LINE : " + str(exc_tb.tb_lineno))
    messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
    print("\n" + typeE +
          "\n" + fileE +
          "\n" + lineE +
          "\n" + messageE)
#}
else: #{
    print("FIN...")
#}

FIN...


In [4]:
# CREATE TIME STAMP (string) FOR ZIP FOLDER
ts = pd.Timestamp.now()
ts_str = str(ts)
ts_str = ts_str[:10]
print(ts_str)

2019-07-26


In [5]:
# DIRECTORIES TO SCAN
in_directory_1 = "F:/APPS/CofA/"
in_directory_2 = "G:/C of A's/#Email Node/"

In [6]:
out_directory = "."

In [7]:
zip_dir_path = os.path.join(out_directory, ts_str)
print(zip_dir_path)

.\2019-07-26


In [8]:
# if 'zip_folder' DOES NOT YET EXIST!
if not os.path.exists(zip_dir_path): #{
    # MAKE IT EXIST!
    os.makedirs(zip_dir_path)
#}

In [9]:
# RESET/RE-ESTABLISH 'out_directory'
out_directory = Path(zip_dir_path)

In [10]:
watermark = "C:/data/inbound/Agilent_CofA_Letterhead_03-21-19.pdf"

## *DEFINE FUNCTIONS*

In [11]:
def create_watermark(input_pdf, output, watermark): #{
    try: #{
        watermark_obj = PdfFileReader(watermark)
        watermark_page = watermark_obj.getPage(0)
        
        pdf_reader = PdfFileReader(input_pdf)
        pdf_writer = PdfFileWriter()
        
        # Watermark all the pages
        for page in range(pdf_reader.getNumPages()): #{
            page = pdf_reader.getPage(page)
            page.mergePage(watermark_page)
            pdf_writer.addPage(page)
        #}
        
        with open(output, 'wb') as out: #{
            pdf_writer.write(out)
        #}
    #}
    except: #{
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        logging.info("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
    #}
    else: #{
        logging.info("[Watermark-PDF] FIN...")
    #}
    return
#}

In [12]:
def get_all_file_paths(directory): #{
    
    # initializing empty file paths list
    file_paths = []
    
    # crawling through directory and subdirectories
    for root, directories, files in os.walk(directory): #{
        for filename in files: #{
            # join the two strings in order to form the full filepath
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
        #}
    #}
    
    # returning all file paths
    return file_paths

#}

In [13]:
"""
TAKES IN THE PATH OF A FILE AND DETERMINES:
is modification or created date older? 
returns the more accurate information 
"""
def pull_creation_modification_date(the_file_path): #{
    # GET CREATION DATE
    file_date = os.path.getctime(pdf_path)
    # GET CREATION_DATE
    created = os.stat(pdf_path).st_ctime_ns
    # CONVERT CREATION_DATE TO PANDAS.TIMESTAMP
    c_stamp = pd.Timestamp(created)
    # GET MODIFICATION_TIME
    modified = os.stat(pdf_path).st_mtime_ns
    # CONVERT MODIFIED TIME TO PANDAS.TIMESTAMP
    m_stamp = pd.Timestamp(modified)
    #########################################3##
    # PANDAS.TIMEDELTA FOR SUBTRACTING DATES
    creation_delta = pd.Timedelta(created - modified)
    logging.debug("DELTA TIME (created - modified) >>> " + str(creation_delta))
    # CREATE CHECK DELTA TO USE 'AGAINST' CREATION_DELTA
    check_delta = pd.Timedelta(value=1, unit='min')
    # IF DELTA IS LESS THAN ONE-MINUTE (NEGATIVE)
    if (creation_delta <= check_delta): #{
        logging.debug("<< CREATION DATE IS OLDER >>")
        logging.debug("CREATED == " + str(c_stamp)) # WAS: logging.info
        logging.debug("MODIFIED == " + str(m_stamp))
        # SET STRING VAR FOR EMAIL OUTPUT
        date_str = str(c_stamp)
    #}
    # ELSE... DELTA IS GREATER THAN ONE-MINUTE
    else: #{
        logging.debug("<< MODIFIED DATE IS OLDER >>")
        logging.debug("MODIFIED == " + str(m_stamp)) # WAS: logging.info
        logging.debug("CREATED == " + str(c_stamp))
        # SET STRING VAR FOR EMAIL OUTPUT
        date_str = str(m_stamp)
    #}
    return date_str
#}

In [14]:
in_file = "batch_list.csv"

In [15]:
df_batch = pd.read_csv(in_file, header=None, names=['Lot_No'], engine='python')

In [16]:
df_batch.head(5)

,Lot_No
0,47995N-1
1,DWM-544-1
2,DWM-580-1
3,DWM-583-1
4,DWM-588-1


In [17]:
test_glob = str(in_directory_1 + "*")
print(test_glob)

F:/APPS/CofA/*


In [18]:
# TOTAL counter
x = 0

for lot_no in df_batch.itertuples(): #{
    logging.info("======== " + str(lot_no[1]) + " ========")
    # CHECK AND CREATE NEW DIR PATH
    new_dir_path = os.path.join(out_directory, str(lot_no[1]))
    # IF 'lot_no' DIRECTORY DOES NOT EXIST:
    if not os.path.exists(new_dir_path): #{
        # MAKE IT EXIST!
        os.makedirs(new_dir_path)
    #}
    # CREATE TEMP DIR/FOLDER TO WORK INSIDE OF:
    with tempfile.TemporaryDirectory() as directory_name: #{
        # LOCAL COUNTER
        j = 0
        the_dir = Path(directory_name)
        logging.info("TEMP_DIR== " + str(the_dir))
        # create variable to hold most recent DATETIME
        newest_date = pd.Timestamp.today()
        # create str to hold our most recent file (path)
        newest_cofa = ""
        # GLOB AND ITERATURE THROUGH EACH FILE MATCHING NAME ETC
        for name in sorted(glob.glob(in_directory_1 + "*" + str(lot_no[1]) + "*")): #{
            # CREATE PATH VARIABLE
            pdf_path = Path(str(name))
            ###########################################
            the_date_str = pull_creation_modification_date(pdf_path)
            """
            the_c_date = pd.Timedelta(the_date_str)
            if newest_date > the_c_date: #{
                # THE C_DATE IS OLDER
                newest_date = newest_date
            #}
            else: #{
                # THE "LAST" C_DATE IS OLDER
                # RESET 
                newest_date = the_c_date
            #}
            """
            #####################################################
            logging.info("FILE CREATED == " + the_date_str)
            logging.info("THE_FILE_PATH == " + str(pdf_path))
            logging.info("FILE_EXISTS == " + str(os.path.exists(pdf_path)))
            # GET BASE NAME (FILE NAME)
            file_name = os.path.basename(pdf_path)
            # CREATE TEMP PATH
            temp_path = os.path.join(the_dir, file_name)
            # COPY FILE TO NEW LOCATION
            shutil.copy2(pdf_path, temp_path)
            # CREATE WATERMARK ON NEW FILE
            # (OUTPUT == out_directory + str(lot_no[1]) + filename)
            create_watermark(input_pdf=temp_path,
                            output=os.path.join(new_dir_path, file_name),
                            watermark=watermark)
            # increase counts
            x += 1
            j += 1
            logging.info("LOCAL count == " + str(j))
        #}
    #}
    logging.info("TOTAL count == " + (str(x)))
    logging.info("Directory exists after? " + str(the_dir.exists()))
    logging.info("Contents after: " + str(list(the_dir.glob("*"))))
    logging.info("told you so")
#}

2019-07-26-134138 : ======== 47995N-1 ========
2019-07-26-134138 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmp7nfynoou
2019-07-26-134139 : TOTAL count == 0
2019-07-26-134139 : Directory exists after? False
2019-07-26-134139 : Contents after: []
2019-07-26-134139 : told you so
2019-07-26-134139 : ======== DWM-544-1 ========
2019-07-26-134139 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmpuaxh5g3j
2019-07-26-134141 : FILE CREATED == 2019-04-17 15:45:44.568289900
2019-07-26-134141 : THE_FILE_PATH == F:\APPS\CofA\DWM-544-1@0006460286.pdf
2019-07-26-134141 : FILE_EXISTS == True
2019-07-26-134141 : [Watermark-PDF] FIN...
2019-07-26-134141 : LOCAL count == 1
2019-07-26-134141 : FILE CREATED == 2019-06-26 20:59:57.088455600
2019-07-26-134141 : THE_FILE_PATH == F:\APPS\CofA\DWM-544-1@0006476293.pdf
2019-07-26-134141 : FILE_EXISTS == True
2019-07-26-134141 : [Watermark-PDF] FIN...
2019-07-26-134141 : LOCAL count == 2
2019-07-26-134141 : TOTAL count == 2
2019-07-26-134141 : Director

2019-07-26-134202 : THE_FILE_PATH == F:\APPS\CofA\IBF-1245@CT-0400.pdf
2019-07-26-134202 : FILE_EXISTS == True
2019-07-26-134202 : [Watermark-PDF] FIN...
2019-07-26-134202 : LOCAL count == 16
2019-07-26-134202 : FILE CREATED == 2019-03-12 12:52:34.773755500
2019-07-26-134202 : THE_FILE_PATH == F:\APPS\CofA\IBF-1245@CT-1199.pdf
2019-07-26-134202 : FILE_EXISTS == True
2019-07-26-134203 : [Watermark-PDF] FIN...
2019-07-26-134203 : LOCAL count == 17
2019-07-26-134203 : TOTAL count == 22
2019-07-26-134203 : Directory exists after? False
2019-07-26-134203 : Contents after: []
2019-07-26-134203 : told you so
2019-07-26-134203 : ======== ICP-042 ========
2019-07-26-134203 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmp61kxwvvx
2019-07-26-134204 : FILE CREATED == 2015-03-17 15:28:52.008451600
2019-07-26-134204 : THE_FILE_PATH == F:\APPS\CofA\ICP-042@CL-4904.pdf
2019-07-26-134204 : FILE_EXISTS == True
2019-07-26-134205 : [Watermark-PDF] FIN...
2019-07-26-134205 : LOCAL count == 1
2019-07-2

2019-07-26-134220 : TOTAL count == 49
2019-07-26-134220 : Directory exists after? False
2019-07-26-134220 : Contents after: []
2019-07-26-134220 : told you so
2019-07-26-134220 : ======== ICP-125 ========
2019-07-26-134220 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmp_9xbx1k4
2019-07-26-134221 : FILE CREATED == 2019-06-03 18:13:15.118574100
2019-07-26-134221 : THE_FILE_PATH == F:\APPS\CofA\ICP-125@0006470502.pdf
2019-07-26-134221 : FILE_EXISTS == True
2019-07-26-134222 : [Watermark-PDF] FIN...
2019-07-26-134222 : LOCAL count == 1
2019-07-26-134222 : FILE CREATED == 2015-05-12 10:40:07.798819800
2019-07-26-134222 : THE_FILE_PATH == F:\APPS\CofA\ICP-125@CM-2432.pdf
2019-07-26-134222 : FILE_EXISTS == True
2019-07-26-134223 : [Watermark-PDF] FIN...
2019-07-26-134223 : LOCAL count == 2
2019-07-26-134223 : FILE CREATED == 2018-04-09 16:23:45.626666300
2019-07-26-134223 : THE_FILE_PATH == F:\APPS\CofA\ICP-125@CS-1643.pdf
2019-07-26-134223 : FILE_EXISTS == True
2019-07-26-134223 : [Wat

2019-07-26-134239 : [Watermark-PDF] FIN...
2019-07-26-134239 : LOCAL count == 2
2019-07-26-134239 : FILE CREATED == 2018-04-05 16:07:52.054406500
2019-07-26-134239 : THE_FILE_PATH == F:\APPS\CofA\ICP-333@AU-01246.pdf
2019-07-26-134239 : FILE_EXISTS == True
2019-07-26-134239 : [Watermark-PDF] FIN...
2019-07-26-134239 : LOCAL count == 3
2019-07-26-134239 : TOTAL count == 79
2019-07-26-134239 : Directory exists after? False
2019-07-26-134239 : Contents after: []
2019-07-26-134239 : told you so
2019-07-26-134239 : ======== ICP-356 ========
2019-07-26-134239 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmpl0qwgz5b
2019-07-26-134241 : FILE CREATED == 2015-03-30 11:47:51.073000
2019-07-26-134241 : THE_FILE_PATH == F:\APPS\CofA\ICP-356@AU-00033.pdf
2019-07-26-134241 : FILE_EXISTS == True
2019-07-26-134242 : [Watermark-PDF] FIN...
2019-07-26-134242 : LOCAL count == 1
2019-07-26-134242 : FILE CREATED == 2015-08-25 14:20:14.326384
2019-07-26-134242 : THE_FILE_PATH == F:\APPS\CofA\ICP-356@AU-

2019-07-26-134248 : FILE CREATED == 2017-02-03 16:54:50.318023400
2019-07-26-134248 : THE_FILE_PATH == F:\APPS\CofA\IMS-102@CR-0285.pdf
2019-07-26-134248 : FILE_EXISTS == True
2019-07-26-134249 : [Watermark-PDF] FIN...
2019-07-26-134249 : LOCAL count == 4
2019-07-26-134249 : FILE CREATED == 2018-03-27 22:04:08.765326100
2019-07-26-134249 : THE_FILE_PATH == F:\APPS\CofA\IMS-102@CS-1404.pdf
2019-07-26-134249 : FILE_EXISTS == True
2019-07-26-134249 : [Watermark-PDF] FIN...
2019-07-26-134249 : LOCAL count == 5
2019-07-26-134249 : FILE CREATED == 2015-05-14 17:32:17.472261300
2019-07-26-134249 : THE_FILE_PATH == F:\APPS\CofA\IMS-102@M00891.pdf
2019-07-26-134249 : FILE_EXISTS == True
2019-07-26-134250 : [Watermark-PDF] FIN...
2019-07-26-134250 : LOCAL count == 6
2019-07-26-134250 : FILE CREATED == 2015-05-14 17:33:23.191431900
2019-07-26-134250 : THE_FILE_PATH == F:\APPS\CofA\IMS-102@P00833.pdf
2019-07-26-134250 : FILE_EXISTS == True
2019-07-26-134250 : [Watermark-PDF] FIN...
2019-07-26-1342

...

2019-07-26-134549 : LOCAL count == 3
2019-07-26-134549 : FILE CREATED == 2014-10-06 17:50:36.449026100
2019-07-26-134549 : THE_FILE_PATH == F:\APPS\CofA\RPC-164AS@CL-3988.PDF
2019-07-26-134549 : FILE_EXISTS == True
2019-07-26-134549 : [Watermark-PDF] FIN...
2019-07-26-134549 : LOCAL count == 4
2019-07-26-134549 : FILE CREATED == 2016-10-27 12:54:57.347315700
2019-07-26-134549 : THE_FILE_PATH == F:\APPS\CofA\RPC-164AS@CP-5366.PDF
2019-07-26-134550 : FILE_EXISTS == True
2019-07-26-134550 : [Watermark-PDF] FIN...
2019-07-26-134550 : LOCAL count == 5
2019-07-26-134550 : FILE CREATED == 2018-11-07 13:10:58.408492800
2019-07-26-134550 : THE_FILE_PATH == F:\APPS\CofA\RPC-164AS@CS-5381.PDF
2019-07-26-134550 : FILE_EXISTS == True
2019-07-26-134550 : [Watermark-PDF] FIN...
2019-07-26-134550 : LOCAL count == 6
2019-07-26-134550 : TOTAL count == 426
2019-07-26-134550 : Directory exists after? False
2019-07-26-134550 : Contents after: []
2019-07-26-134550 : told you so
2019-07-26-134550 : ======== 

2019-07-26-134624 : TOTAL count == 436
2019-07-26-134624 : Directory exists after? False
2019-07-26-134624 : Contents after: []
2019-07-26-134624 : told you so
2019-07-26-134624 : ======== UST-200-1 ========
2019-07-26-134624 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmpcdbgewlt
2019-07-26-134628 : TOTAL count == 436
2019-07-26-134628 : Directory exists after? False
2019-07-26-134628 : Contents after: []
2019-07-26-134628 : told you so
2019-07-26-134628 : ======== WRK-151 ========
2019-07-26-134628 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmplq5kzsme
2019-07-26-134631 : FILE CREATED == 2016-03-07 20:10:07.380101900
2019-07-26-134631 : THE_FILE_PATH == F:\APPS\CofA\WRK-151A@NT053304.pdf
2019-07-26-134631 : FILE_EXISTS == True
2019-07-26-134632 : [Watermark-PDF] FIN...
2019-07-26-134632 : LOCAL count == 1
2019-07-26-134632 : FILE CREATED == 2017-06-06 13:23:52.816825600
2019-07-26-134632 : THE_FILE_PATH == F:\APPS\CofA\WRK-151A@NT057373.pdf
2019-07-26-134632 : FILE_EXIST

2019-07-26-134642 : FILE_EXISTS == True
2019-07-26-134643 : [Watermark-PDF] FIN...
2019-07-26-134643 : LOCAL count == 30
2019-07-26-134643 : FILE CREATED == 2017-06-05 15:43:22.419356500
2019-07-26-134643 : THE_FILE_PATH == F:\APPS\CofA\WRK-151O@NT057361.pdf
2019-07-26-134643 : FILE_EXISTS == True
2019-07-26-134643 : [Watermark-PDF] FIN...
2019-07-26-134643 : LOCAL count == 31
2019-07-26-134643 : TOTAL count == 467
2019-07-26-134643 : Directory exists after? False
2019-07-26-134643 : Contents after: []
2019-07-26-134643 : told you so
2019-07-26-134643 : ======== WRK-151A ========
2019-07-26-134643 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmplne3pcoa
2019-07-26-134645 : FILE CREATED == 2016-03-07 20:10:07.380101900
2019-07-26-134645 : THE_FILE_PATH == F:\APPS\CofA\WRK-151A@NT053304.pdf
2019-07-26-134645 : FILE_EXISTS == True
2019-07-26-134645 : [Watermark-PDF] FIN...
2019-07-26-134645 : LOCAL count == 1
2019-07-26-134645 : FILE CREATED == 2017-06-06 13:23:52.816825600
2019-07-2

2019-07-26-134709 : THE_FILE_PATH == F:\APPS\CofA\WRK-151J@NT057356.pdf
2019-07-26-134709 : FILE_EXISTS == True
2019-07-26-134709 : [Watermark-PDF] FIN...
2019-07-26-134709 : LOCAL count == 2
2019-07-26-134709 : TOTAL count == 487
2019-07-26-134709 : Directory exists after? False
2019-07-26-134709 : Contents after: []
2019-07-26-134709 : told you so
2019-07-26-134709 : ======== WRK-151K ========
2019-07-26-134709 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmpvilatt0n
2019-07-26-134711 : FILE CREATED == 2016-03-07 21:32:45.179849100
2019-07-26-134711 : THE_FILE_PATH == F:\APPS\CofA\WRK-151K@NT053313.pdf
2019-07-26-134711 : FILE_EXISTS == True
2019-07-26-134711 : [Watermark-PDF] FIN...
2019-07-26-134711 : LOCAL count == 1
2019-07-26-134711 : FILE CREATED == 2017-06-05 15:27:45.510743200
2019-07-26-134711 : THE_FILE_PATH == F:\APPS\CofA\WRK-151K@NT057357.pdf
2019-07-26-134711 : FILE_EXISTS == True
2019-07-26-134711 : [Watermark-PDF] FIN...
2019-07-26-134711 : LOCAL count == 2
2019-

2019-07-26-134744 : TOTAL count == 498
2019-07-26-134744 : Directory exists after? False
2019-07-26-134744 : Contents after: []
2019-07-26-134744 : told you so
2019-07-26-134744 : ======== PP-301-1 ========
2019-07-26-134744 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmp37m0h1nl
2019-07-26-134745 : TOTAL count == 498
2019-07-26-134745 : Directory exists after? False
2019-07-26-134745 : Contents after: []
2019-07-26-134745 : told you so
2019-07-26-134745 : ======== PP-311-1 ========
2019-07-26-134745 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmpq461t6fo
2019-07-26-134747 : TOTAL count == 498
2019-07-26-134747 : Directory exists after? False
2019-07-26-134747 : Contents after: []
2019-07-26-134747 : told you so
2019-07-26-134747 : ======== PP-341-1 ========
2019-07-26-134747 : TEMP_DIR== C:\Users\derbates\AppData\Local\Temp\tmp08t6nqb6
2019-07-26-134748 : TOTAL count == 498
2019-07-26-134748 : Directory exists after? False
2019-07-26-134748 : Contents after: []
2019-07-26-

## *`os.walk()` thru WORKING DIRECTORY* 

## and keep only **MOST RECENT** file 

## `or`

## keep only **MOST OLDEST** file

In [71]:
most_recent = False

In [72]:
type(most_recent)

bool

In [73]:
if most_recent is True: #{
    print("true")
#}
else: #{
    print("false!")
#}

false!


In [19]:
in_directory = "."

In [20]:
# CREATE LIST VARIABLES THAT WILL BECOME COLS IN DATAFRAME
file_col = []
date_col = []

---
*PROCESS FOR RETURNING `MOST RECENT`*

In [80]:
# create LIST vars to hold all the most recent files/dates
recent_files = []
recent_dates = []

In [82]:
# iterate thru directory/directories 
for root, d_names, f_names in os.walk(in_directory): #{
    logging.info("===================== START ====================== \n")
    logging.info("\t *root* = " + str(root) + "\n") 
    logging.info("\t *dict* = " + str(d_names) + "\n")
    if len(f_names) >= 0: #{
        logging.info("\t *file(s)*: \n")
        # CREATE VARS for "most_recents"
        most_recent_date = pd.Timestamp(year=1999, month=12, day=31)
        most_recent_name = "test.pdf"
        # multiple file(s)
        for f in f_names: #{
            logging.info("\t FILE-NAME == " + str(f) + "\n")
            file_name = os.path.basename(f)
            # add **file NAME** to list
            file_col.append(file_name)
            # create variable for *ORIGINAL* file_path
            f_path = os.path.join("F:/APPS/CofA/", file_name)
            logging.info("\t F_PATH == " + str(f_path) + "\n")
            if os.path.exists(f_path): #{
                # create date var
                date = os.path.getctime(f_path)
                # FORMAT date var & SET AS str
                date_str = str(datetime.fromtimestamp(date))
                # set our date STR to a Pandas.TIMESTAMP var
                ts = pd.Timestamp(ts_input=date_str)
                logging.info("\t CREATE_DATE == " + str(ts) + "\n")
                ##################################################### 
                # CHECK IF MOST RECENT (if 'ts' is more recent...)
                if ts > most_recent_date: #{
                    # RE_ASSIGN 'most_recent_date'
                    most_recent_date = ts
                    most_recent_name = f
                    logging.info("\t most_recent_date == " + str(most_recent_date) + "\n")
                    logging.info("\t most_recent_name == " + str(most_recent_name) + "\n")
                #}
                else: #{
                    # keep "most_recent_date" the same
                    logging.info("\t most_recent_date == " + str(most_recent_date) + "\n")
                    logging.info("\t most_recent_name == " + str(most_recent_name) + "\n")
                #}
                #logging.info("\t ts == " + str(ts) + "\n")
                # add **file DATE** to list
                date_col.append(date_str)
            #}
            else: #{
                logging.info("\t CREATE_DATE == NaN (doesn't exist) \n")
            #}
        #}
    #}
    else: #{
        # no files
        logging.info("\t NO-FILES!")
    #}
    logging.info("=====================  END  ======================")
    recent_files.append(most_recent_name)
    recent_dates.append(most_recent_date)
#}

2019-07-26-164308 : ===================== START ====================== 

2019-07-26-164308 : 	 *root* = .

2019-07-26-164308 : 	 *dict* = ['.ipynb_checkpoints', '2019-07-25', '2019-07-26']

2019-07-26-164308 : 	 *file(s)*: 

2019-07-26-164308 : 	 FILE-NAME == batch_list.csv

2019-07-26-164308 : 	 F_PATH == F:/APPS/CofA/batch_list.csv

2019-07-26-164308 : 	 CREATE_DATE == NaN (doesn't exist) 

2019-07-26-164308 : 	 FILE-NAME == CC_2019_07_25.ipynb

2019-07-26-164308 : 	 F_PATH == F:/APPS/CofA/CC_2019_07_25.ipynb

2019-07-26-164308 : 	 CREATE_DATE == NaN (doesn't exist) 

2019-07-26-164308 : 	 FILE-NAME == Remaining_CofAs.xlsx

2019-07-26-164308 : 	 F_PATH == F:/APPS/CofA/Remaining_CofAs.xlsx

2019-07-26-164308 : 	 CREATE_DATE == NaN (doesn't exist) 

2019-07-26-164308 : =====================  END  ======================
2019-07-26-164308 : ===================== START ====================== 

2019-07-26-164308 : 	 *root* = .\.ipynb_checkpoints

2019-07-26-164308 : 	 *dict* = []

2019-07-

2019-07-26-164308 : 	 FILE-NAME == IBF-1245@CM-2198.pdf

2019-07-26-164308 : 	 F_PATH == F:/APPS/CofA/IBF-1245@CM-2198.pdf

2019-07-26-164308 : 	 CREATE_DATE == 2015-05-04 08:42:37.966016

2019-07-26-164308 : 	 most_recent_date == 2015-05-04 08:42:37.966016

2019-07-26-164308 : 	 most_recent_name == IBF-1245@CM-2198.pdf

2019-07-26-164308 : 	 FILE-NAME == IBF-1245@CM-4282.pdf

2019-07-26-164308 : 	 F_PATH == F:/APPS/CofA/IBF-1245@CM-4282.pdf

2019-07-26-164308 : 	 CREATE_DATE == 2015-08-11 14:47:54.769735

2019-07-26-164308 : 	 most_recent_date == 2015-08-11 14:47:54.769735

2019-07-26-164308 : 	 most_recent_name == IBF-1245@CM-4282.pdf

2019-07-26-164308 : 	 FILE-NAME == IBF-1245@CM-5863.pdf

2019-07-26-164308 : 	 F_PATH == F:/APPS/CofA/IBF-1245@CM-5863.pdf

2019-07-26-164308 : 	 CREATE_DATE == 2015-11-12 13:49:12.606382

2019-07-26-164308 : 	 most_recent_date == 2015-11-12 13:49:12.606382

2019-07-26-164308 : 	 most_recent_name == IBF-1245@CM-5863.pdf

2019-07-26-164308 : 	 FILE-NAME

2019-07-26-164308 : 	 FILE-NAME == ICP-048@CP-5285.pdf

2019-07-26-164308 : 	 F_PATH == F:/APPS/CofA/ICP-048@CP-5285.pdf

2019-07-26-164308 : 	 CREATE_DATE == 2016-10-25 06:50:05.473080

2019-07-26-164308 : 	 most_recent_date == 2016-10-25 06:50:05.473080

2019-07-26-164308 : 	 most_recent_name == ICP-048@CP-5285.pdf

2019-07-26-164308 : 	 FILE-NAME == ICP-048@J00298.pdf

2019-07-26-164308 : 	 F_PATH == F:/APPS/CofA/ICP-048@J00298.pdf

2019-07-26-164308 : 	 CREATE_DATE == 2015-04-16 09:36:28.861186

2019-07-26-164308 : 	 most_recent_date == 2016-10-25 06:50:05.473080

2019-07-26-164308 : 	 most_recent_name == ICP-048@CP-5285.pdf

2019-07-26-164308 : 	 FILE-NAME == ICP-048@K00951.pdf

2019-07-26-164308 : 	 F_PATH == F:/APPS/CofA/ICP-048@K00951.pdf

2019-07-26-164308 : 	 CREATE_DATE == 2015-04-16 09:37:06.189311

2019-07-26-164308 : 	 most_recent_date == 2016-10-25 06:50:05.473080

2019-07-26-164308 : 	 most_recent_name == ICP-048@CP-5285.pdf

2019-07-26-164308 : 	 FILE-NAME == ICP-048@M

2019-07-26-164309 : 	 FILE-NAME == ICP-125@K00953.pdf

2019-07-26-164309 : 	 F_PATH == F:/APPS/CofA/ICP-125@K00953.pdf

2019-07-26-164309 : 	 CREATE_DATE == 2015-04-23 14:19:29.484901

2019-07-26-164309 : 	 most_recent_date == 2019-06-03 14:20:36.381002

2019-07-26-164309 : 	 most_recent_name == ICP-125@0006470502.pdf

2019-07-26-164309 : 	 FILE-NAME == ICP-125@M00276.pdf

2019-07-26-164309 : 	 F_PATH == F:/APPS/CofA/ICP-125@M00276.pdf

2019-07-26-164309 : 	 CREATE_DATE == 2015-03-25 16:06:45.482415

2019-07-26-164309 : 	 most_recent_date == 2019-06-03 14:20:36.381002

2019-07-26-164309 : 	 most_recent_name == ICP-125@0006470502.pdf

2019-07-26-164309 : 	 FILE-NAME == ICP-125@M00276B.pdf

2019-07-26-164309 : 	 F_PATH == F:/APPS/CofA/ICP-125@M00276B.pdf

2019-07-26-164309 : 	 CREATE_DATE == 2015-05-05 10:26:56.414486

2019-07-26-164309 : 	 most_recent_date == 2019-06-03 14:20:36.381002

2019-07-26-164309 : 	 most_recent_name == ICP-125@0006470502.pdf

2019-07-26-164309 : 	 FILE-NAME == 

2019-07-26-164309 : 	 most_recent_name == ICP-333@AU-01246.pdf

2019-07-26-164309 : =====================  END  ======================
2019-07-26-164309 : ===================== START ====================== 

2019-07-26-164309 : 	 *root* = .\2019-07-25\ICP-356

2019-07-26-164309 : 	 *dict* = []

2019-07-26-164309 : 	 *file(s)*: 

2019-07-26-164309 : 	 FILE-NAME == ICP-356@AU-00033.pdf

2019-07-26-164309 : 	 F_PATH == F:/APPS/CofA/ICP-356@AU-00033.pdf

2019-07-26-164309 : 	 CREATE_DATE == 2015-03-30 07:47:51.073000

2019-07-26-164309 : 	 most_recent_date == 2015-03-30 07:47:51.073000

2019-07-26-164309 : 	 most_recent_name == ICP-356@AU-00033.pdf

2019-07-26-164309 : 	 FILE-NAME == ICP-356@AU-00894.pdf

2019-07-26-164309 : 	 F_PATH == F:/APPS/CofA/ICP-356@AU-00894.pdf

2019-07-26-164309 : 	 CREATE_DATE == 2015-08-25 10:20:14.326384

2019-07-26-164309 : 	 most_recent_date == 2015-08-25 10:20:14.326384

2019-07-26-164309 : 	 most_recent_name == ICP-356@AU-00894.pdf

2019-07-26-164309 : 	 F

...

2019-07-26-164321 : 	 CREATE_DATE == 2016-05-12 15:17:16.898369

2019-07-26-164321 : 	 most_recent_date == 2019-05-31 07:42:04.033626

2019-07-26-164321 : 	 most_recent_name == RPC-133S@0006469196.pdf

2019-07-26-164321 : 	 FILE-NAME == RPC-133S@CP-2307.PDF

2019-07-26-164321 : 	 F_PATH == F:/APPS/CofA/RPC-133S@CP-2307.PDF

2019-07-26-164321 : 	 CREATE_DATE == 2016-05-16 09:07:14.496444

2019-07-26-164321 : 	 most_recent_date == 2019-05-31 07:42:04.033626

2019-07-26-164321 : 	 most_recent_name == RPC-133S@0006469196.pdf

2019-07-26-164321 : =====================  END  ======================
2019-07-26-164321 : ===================== START ====================== 

2019-07-26-164321 : 	 *root* = .\2019-07-26\RPC-156AS

2019-07-26-164321 : 	 *dict* = []

2019-07-26-164321 : 	 *file(s)*: 

2019-07-26-164321 : 	 FILE-NAME == RPC-156AS@CE-0680.pdf

2019-07-26-164321 : 	 F_PATH == F:/APPS/CofA/RPC-156AS@CE-0680.pdf

2019-07-26-164321 : 	 CREATE_DATE == 2010-06-30 15:03:03.964614

2019-07-26-1

2019-07-26-164322 : 	 *dict* = []

2019-07-26-164322 : 	 *file(s)*: 

2019-07-26-164322 : =====================  END  ======================
2019-07-26-164322 : ===================== START ====================== 

2019-07-26-164322 : 	 *root* = .\2019-07-26\STM-262-1

2019-07-26-164322 : 	 *dict* = []

2019-07-26-164322 : 	 *file(s)*: 

2019-07-26-164322 : =====================  END  ======================
2019-07-26-164322 : ===================== START ====================== 

2019-07-26-164322 : 	 *root* = .\2019-07-26\STM-320N-1

2019-07-26-164322 : 	 *dict* = []

2019-07-26-164322 : 	 *file(s)*: 

2019-07-26-164322 : =====================  END  ======================
2019-07-26-164322 : ===================== START ====================== 

2019-07-26-164322 : 	 *root* = .\2019-07-26\STM-520-1

2019-07-26-164322 : 	 *dict* = []

2019-07-26-164322 : 	 *file(s)*: 

2019-07-26-164322 : =====================  END  ======================
2019-07-26-164322 : ===================== START ===

2019-07-26-164322 : 	 F_PATH == F:/APPS/CofA/WRK-151D@NT053307.pdf

2019-07-26-164322 : 	 CREATE_DATE == 2019-05-28 10:03:48.750930

2019-07-26-164322 : 	 most_recent_date == 2019-05-28 10:03:49.204060

2019-07-26-164322 : 	 most_recent_name == WRK-151A@NT053304.pdf

2019-07-26-164322 : 	 FILE-NAME == WRK-151D@NT057350.pdf

2019-07-26-164322 : 	 F_PATH == F:/APPS/CofA/WRK-151D@NT057350.pdf

2019-07-26-164322 : 	 CREATE_DATE == 2019-05-28 09:54:44.025207

2019-07-26-164322 : 	 most_recent_date == 2019-05-28 10:03:49.204060

2019-07-26-164322 : 	 most_recent_name == WRK-151A@NT053304.pdf

2019-07-26-164322 : 	 FILE-NAME == WRK-151E@NT053308.pdf

2019-07-26-164322 : 	 F_PATH == F:/APPS/CofA/WRK-151E@NT053308.pdf

2019-07-26-164322 : 	 CREATE_DATE == 2019-05-28 10:03:48.641553

2019-07-26-164322 : 	 most_recent_date == 2019-05-28 10:03:49.204060

2019-07-26-164322 : 	 most_recent_name == WRK-151A@NT053304.pdf

2019-07-26-164322 : 	 FILE-NAME == WRK-151E@NT057351.pdf

2019-07-26-164322 : 	 

2019-07-26-164322 : 	 *root* = .\2019-07-26\WRK-151A

2019-07-26-164322 : 	 *dict* = []

2019-07-26-164322 : 	 *file(s)*: 

2019-07-26-164322 : 	 FILE-NAME == WRK-151A@NT053304.pdf

2019-07-26-164322 : 	 F_PATH == F:/APPS/CofA/WRK-151A@NT053304.pdf

2019-07-26-164322 : 	 CREATE_DATE == 2019-05-28 10:03:49.204060

2019-07-26-164322 : 	 most_recent_date == 2019-05-28 10:03:49.204060

2019-07-26-164322 : 	 most_recent_name == WRK-151A@NT053304.pdf

2019-07-26-164322 : 	 FILE-NAME == WRK-151A@NT057373.pdf

2019-07-26-164322 : 	 F_PATH == F:/APPS/CofA/WRK-151A@NT057373.pdf

2019-07-26-164322 : 	 CREATE_DATE == 2019-05-28 09:54:42.087682

2019-07-26-164322 : 	 most_recent_date == 2019-05-28 10:03:49.204060

2019-07-26-164322 : 	 most_recent_name == WRK-151A@NT053304.pdf

2019-07-26-164322 : =====================  END  ======================
2019-07-26-164322 : ===================== START ====================== 

2019-07-26-164322 : 	 *root* = .\2019-07-26\WRK-151B

2019-07-26-164322 : 	 *dic

2019-07-26-164323 : ===================== START ====================== 

2019-07-26-164323 : 	 *root* = .\2019-07-26\WRK-151J

2019-07-26-164323 : 	 *dict* = []

2019-07-26-164323 : 	 *file(s)*: 

2019-07-26-164323 : 	 FILE-NAME == WRK-151J@NT053316.pdf

2019-07-26-164323 : 	 F_PATH == F:/APPS/CofA/WRK-151J@NT053316.pdf

2019-07-26-164323 : 	 CREATE_DATE == 2019-05-28 10:03:47.813417

2019-07-26-164323 : 	 most_recent_date == 2019-05-28 10:03:47.813417

2019-07-26-164323 : 	 most_recent_name == WRK-151J@NT053316.pdf

2019-07-26-164323 : 	 FILE-NAME == WRK-151J@NT057356.pdf

2019-07-26-164323 : 	 F_PATH == F:/APPS/CofA/WRK-151J@NT057356.pdf

2019-07-26-164323 : 	 CREATE_DATE == 2019-05-28 09:54:43.228322

2019-07-26-164323 : 	 most_recent_date == 2019-05-28 10:03:47.813417

2019-07-26-164323 : 	 most_recent_name == WRK-151J@NT053316.pdf

2019-07-26-164323 : =====================  END  ======================
2019-07-26-164323 : ===================== START ====================== 

2019-07

In [83]:
# CREATE EMPTY DATAFRAME
df_dates = pd.DataFrame()

In [84]:
# CREATE SERIES OFF LIST
f1 = pd.Series(recent_files)

In [85]:
# CREATE SERIES OFF LIST
d1 = pd.Series(recent_dates)

In [86]:
df_dates['File'] = f1
df_dates['Time'] = d1

In [87]:
df_dates.head(10)

,File,Time
0,test.pdf,1999-12-31 00:00:00.000000
1,test.pdf,1999-12-31 00:00:00.000000
2,test.pdf,1999-12-31 00:00:00.000000
3,test.pdf,1999-12-31 00:00:00.000000
4,DWM-544-1@0006476293.pdf,2019-06-26 16:59:57.088456
5,DWM-580-1@0006472371.pdf,2019-06-10 06:26:02.682714
6,DWM-583-1@CG-3973.PDF,2010-12-21 14:30:59.128316
7,test.pdf,1999-12-31 00:00:00.000000
8,test.pdf,1999-12-31 00:00:00.000000
9,test.pdf,1999-12-31 00:00:00.000000


In [88]:
df_dates.to_csv("most_recent_test.csv", index=False)